Activate the environment and instantiate it if you have not done so yet.

In [1]:
using Pkg
Pkg.activate(".")

"/home/michel/git/structjump-demo/Project.toml"

Load StructJuMP

In [2]:
using StructJuMP

We now create a structured model `m`. At this point the number of second-stage submodels is set, referring to them as *scenarios*.

In [3]:
num_scenarios=2
m = StructuredModel(num_scenarios=num_scenarios)

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

The model is printed and we see it is empty. Like the underlying modeling framework [JuMP](https://github.com/JuliaOpt/JuMP.jl), there are 3 basic components of a model object: `@variable`, `@constraint`, and `@objective`. In addition, nonlinear constraints and objectives are set via `@NLconstraint` and `@NLobjective`.

In [4]:
n=6
@variable(m, x[1:n] >= 0)
@variable(m, y[1:n] >= 0)
for i in 1:n
    @constraint(m, x[i] - y[i] <= i)
end
@NLobjective(m, :Min, x[1] - y[3] + 2x[6] + x[1]*x[2] - 2y[4]*x[3])
m

Minimization problem with:
 * 6 linear constraints
 * 12 variables
Solver is default solver

The above defines the first stage described by `m`. Notice how we can print the model. We now want to create the 2 second-stage models and connect them to the first stage `m`. We create the scenarios using a `for` loop. `StructuredModel` creates a submodel `mk` attached to the parent `m` and with a unique identifier `i`.

In [5]:
for i in 1:num_scenarios
    mk = StructuredModel(parent=m,id=i)
    @variable(mk, w[1:n] >= 0)
    @constraint(mk, sum(i*w[i] for i=2:2:n) == 1)
    @NLconstraint(mk, sum(x[i] - 2w[n-i+1] for i=1:n) <= 0)
    @NLobjective(mk, :Min, w[5] - w[1]*w[1] + 2w[2]*w[3])
end

Let's print again the entire model.

In [6]:
print(m)

Min ((x[1] - y[3]) + 2.0 * x[6] + x[1] * x[2]) - (2.0 * y[4]) * x[3]
Subject to
 x[1] - y[1] ≤ 1
 x[2] - y[2] ≤ 2
 x[3] - y[3] ≤ 3
 x[4] - y[4] ≤ 4
 x[5] - y[5] ≤ 5
 x[6] - y[6] ≤ 6
 x[i] ≥ 0 ∀ i ∈ {1,2,…,5,6}
 y[i] ≥ 0 ∀ i ∈ {1,2,…,5,6}
*** children ***
Child ID 2:
Min (w[5] - w[1] * w[1]) + (2.0 * w[2]) * w[3]
Subject to
 2 w[2] + 4 w[4] + 6 w[6] = 1
 ((x[1] - 2.0 * w[6]) + (x[2] - 2.0 * w[5]) + (x[3] - 2.0 * w[4]) + (x[4] - 2.0 * w[3]) + (x[5] - 2.0 * w[2]) + (x[6] - 2.0 * w[1])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,…,5,6}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[5] ≥ 0
 x[6] ≥ 0
*** children ***

Child ID 1:
Min (w[5] - w[1] * w[1]) + (2.0 * w[2]) * w[3]
Subject to
 2 w[2] + 4 w[4] + 6 w[6] = 1
 ((x[1] - 2.0 * w[6]) + (x[2] - 2.0 * w[5]) + (x[3] - 2.0 * w[4]) + (x[4] - 2.0 * w[3]) + (x[5] - 2.0 * w[2]) + (x[6] - 2.0 * w[1])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,…,5,6}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[5] ≥ 0
 x[6] ≥ 0
*** children ***



We have now created a model. In order to solve it we need to load a solver interface. `StructJuMP` was initially designed as a generic package to run with a variety of solvers. Currently, we support `Ipopt` and the large-scale interior point solver [PIPS](https://github.com/Argonne-National-Laboratory/PIPS). These are loaded via the package `StructJuMPSolverInterface`.

In [7]:
using StructJuMPSolverInterface

In a distributed parallel setting using `MPI`, we can print the local children.

In [8]:
getLocalChildrenIds(m)

2-element Array{Int64,1}:
 1
 2

Here we see both children as we run sequentially. We can access and print the local children at any time.

In [9]:
for i in getLocalChildrenIds(m)
    print(getchildren(m)[i])
end

Min (w[5] - w[1] * w[1]) + (2.0 * w[2]) * w[3]
Subject to
 2 w[2] + 4 w[4] + 6 w[6] = 1
 ((x[1] - 2.0 * w[6]) + (x[2] - 2.0 * w[5]) + (x[3] - 2.0 * w[4]) + (x[4] - 2.0 * w[3]) + (x[5] - 2.0 * w[2]) + (x[6] - 2.0 * w[1])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,…,5,6}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[5] ≥ 0
 x[6] ≥ 0
*** children ***
Min (w[5] - w[1] * w[1]) + (2.0 * w[2]) * w[3]
Subject to
 2 w[2] + 4 w[4] + 6 w[6] = 1
 ((x[1] - 2.0 * w[6]) + (x[2] - 2.0 * w[5]) + (x[3] - 2.0 * w[4]) + (x[4] - 2.0 * w[3]) + (x[5] - 2.0 * w[2]) + (x[6] - 2.0 * w[1])) - 0.0 ≤ 0
 w[i] ≥ 0 ∀ i ∈ {1,2,…,5,6}
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[5] ≥ 0
 x[6] ≥ 0
*** children ***


In [10]:
using Ipopt

In [11]:
solve(m;solver="Ipopt")

┌ Warning: Variable value not defined for x[1]. Check that the model was properly solved.
└ @ JuMP /home/michel/.julia/packages/JuMP/PbnIJ/src/JuMP.jl:448
┌ Warning: Variable value not defined for x[2]. Check that the model was properly solved.
└ @ JuMP /home/michel/.julia/packages/JuMP/PbnIJ/src/JuMP.jl:448
┌ Warning: Variable value not defined for x[3]. Check that the model was properly solved.
└ @ JuMP /home/michel/.julia/packages/JuMP/PbnIJ/src/JuMP.jl:448
┌ Warning: Variable value not defined for x[4]. Check that the model was properly solved.
└ @ JuMP /home/michel/.julia/packages/JuMP/PbnIJ/src/JuMP.jl:448
┌ Warning: Variable value not defined for x[5]. Check that the model was properly solved.
└ @ JuMP /home/michel/.julia/packages/JuMP/PbnIJ/src/JuMP.jl:448
┌ Warning: Variable value not defined for x[6]. Check that the model was properly solved.
└ @ JuMP /home/michel/.julia/packages/JuMP/PbnIJ/src/JuMP.jl:448
┌ Warning: Variable value not defined for y[1]. Check that the model w


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:       36
Number of nonzeros in Lagrangian Hessian.............:       14

Total number of variables............................:       24
                     variables with only lower bounds:       24
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

:Diverging_Iterates